# VIVOS TTS Training Notebook

Notebook này giúp huấn luyện mô hình TTS sử dụng dữ liệu VIVOS trên Google Colab với GPU.

## Cài đặt và chuẩn bị môi trường

Đầu tiên, clone repository từ GitHub và cài đặt các dependencies cần thiết.

In [1]:
# Kiểm tra GPU
!nvidia-smi

Thu Mar 27 09:50:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
# Clone repository từ GitHub
!git clone https://github.com/DangAW2002/vietnamese-tts.git
%cd vietnamese-tts

Cloning into 'vietnamese-tts'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 23 (delta 5), reused 15 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 16.29 KiB | 16.29 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/vietnamese-tts


In [ ]:
# Cài đặt các package cần thiết
!pip install torch torchaudio numpy==2.0.0 numba scipy librosa==0.10.1 matplotlib pandas tensorboard tqdm pyyaml soundfile gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/664.8 MB 118.8 MB/s eta 0:00:05

## Tải dữ liệu đã được xử lý
x
Do kích thước lớn, dữ lixệu VIVOS đã xử lý đcược lưu trữ riêng trên Google Drive. Chúng ta cần tải và giải nén trước khi sử dụng.

In [9]:
import os
import gdown
import zipfile
import shutil

# URL của file zip chứa dữ liệu đã xử lý (cần thay thế bằng link Google Drive thật)
url = "https://drive.google.com/uc?id=1JT3DyD0wnz7-7i4Zf5D9Sp9Q90_iBsga"
zip_path = "vivos_processed.zip"
extract_folder = "data"

# Hỏi người dùng có muốn tải xuống không
download_data = input("Bạn đã cập nhật URL và muốn tải dữ liệu xuống? (yes/no): ")
if download_data.lower() in ["yes", "y"]:
    # Tải file zip
    print("📥 Đang tải dữ liệu VIVOS đã xử lý...")
    gdown.download(url, zip_path, quiet=False, fuzzy=True)

    # Kiểm tra kích thước file
    if os.path.exists(zip_path):
        file_size = os.path.getsize(zip_path) / (1024 * 1024)  # Chuyển sang MB
        print(f"📦 Kích thước file tải về: {file_size:.2f} MB")

        if file_size < 1:  # Kiểm tra nếu file quá nhỏ
            print("⚠ Cảnh báo: File tải xuống có kích thước bất thường, có thể bị lỗi!")

    # Giải nén
    print("📂 Đang giải nén dữ liệu...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print("✅ Hoàn thành tải và giải nén dữ liệu!")

        # Xóa file zip sau khi giải nén
        os.remove(zip_path)

        # 🔄 Xử lý lỗi tên file (nếu có dấu "\")
        for root, _, files in os.walk(extract_folder):
            for file in files:
                if "\\" in file:  # Nếu tên file chứa "\"
                    old_path = os.path.join(root, file)
                    new_name = file.replace("\\", "/")  # Chuyển dấu "\" thành "/"
                    new_path = os.path.join(root, new_name)
                    os.makedirs(os.path.dirname(new_path), exist_ok=True)
                    shutil.move(old_path, new_path)
                    print(f"🔄 Sửa lỗi tên file: {file} → {new_name}")

        # Kiểm tra file đã giải nén
        print("\n📜 Danh sách file đã giải nén:")
        for root, _, files in os.walk(extract_folder):
            for file in files[:10]:  # Hiển thị 10 file đầu tiên
                print(f" - {os.path.join(root, file)}")

    except zipfile.BadZipFile:
        print("❌ Lỗi: File tải xuống không phải là file ZIP hợp lệ!")
else:
    print("🚫 Bỏ qua bước tải dữ liệu. Hãy đảm bảo bạn đã có dữ liệu trong thư mục data/processed.")


Bạn đã cập nhật URL và muốn tải dữ liệu xuống? (yes/no): y
📥 Đang tải dữ liệu VIVOS đã xử lý...


Downloading...
From (original): https://drive.google.com/uc?id=1JT3DyD0wnz7-7i4Zf5D9Sp9Q90_iBsga
From (redirected): https://drive.google.com/uc?id=1JT3DyD0wnz7-7i4Zf5D9Sp9Q90_iBsga&confirm=t&uuid=bc6ed187-5db4-437d-9f95-b548e1f53057
To: /content/vietnamese-tts/vivos_processed.zip
100%|██████████| 1.13G/1.13G [00:05<00:00, 200MB/s]


Streaming output truncated to the last 5000 lines.
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK27_192.pt → processed/train/VIVOSSPK27_192.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK06_T033.pt → processed/train/VIVOSSPK06_T033.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK34_032.pt → processed/train/VIVOSSPK34_032.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK03_R091.pt → processed/train/VIVOSSPK03_R091.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK15_266.pt → processed/train/VIVOSSPK15_266.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK11_R003.pt → processed/train/VIVOSSPK11_R003.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK15_278.pt → processed/train/VIVOSSPK15_278.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK04_R055.pt → processed/train/VIVOSSPK04_R055.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK33_083.pt → processed/train/VIVOSSPK33_083.pt
🔄 Sửa lỗi tên file: processed\train\VIVOSSPK03_R187.pt → processed/train/VIVOSSPK03_R187.pt
🔄 Sửa lỗi tên file: processed\train\VIV

## Huấn luyện mô hình

Bây giờ chúng ta sẽ huấn luyện mô hình TTS sử dụng dữ liệu VIVOS đã được tiền xử lý.

In [7]:
# Kiểm tra dữ liệu đã được tải đúng cách chưa
import os
import glob

train_files = glob.glob("data/processed/train/*.pt")
val_files = glob.glob("data/processed/val/*.pt")
test_files = glob.glob("data/processed/test/*.pt")

print(f"Số file trong tập train: {len(train_files)}")
print(f"Số file trong tập val: {len(val_files)}")
print(f"Số file trong tập test: {len(test_files)}")

if len(train_files) == 0 or len(val_files) == 0 or len(test_files) == 0:
    print("CẢNH BÁO: Không tìm thấy dữ liệu đã xử lý. Hãy quay lại bước trước để tải dữ liệu.")

Số file trong tập train: 0
Số file trong tập val: 0
Số file trong tập test: 0
CẢNH BÁO: Không tìm thấy dữ liệu đã xử lý. Hãy quay lại bước trước để tải dữ liệu.


In [ ]:
# Chạy script training
# Các tham số có thể điều chỉnh:
# --epochs: số epoch
# --batch_size: kích thước batch
# --lr: learning rate

!python src/colab_train.py --epochs 50 --batch_size 64

## Hiển thị quá trình huấn luyện với TensorBoard

In [ ]:
# Cài đặt TensorBoard extension
%load_ext tensorboard
%tensorboard --logdir=checkpoints/logs

## Lưu mô hình đã huấn luyện lên Google Drive

Sau khi huấn luyện xong, bạn nên lưu mô hình lên Google Drive để tránh mất kết quả khi phiên Colab kết thúc.

In [ ]:
# Mount Google Drive để lưu mô hình
from google.colab import drive
drive.mount('/content/drive')

# Nén thư mục checkpoints
!zip -r checkpoints.zip checkpoints/

# Copy file nén vào Google Drive
import os
import shutil

# Tạo thư mục để lưu trữ mô hình trên Drive nếu chưa có
os.makedirs("/content/drive/My Drive/vietnamese-tts-models", exist_ok=True)

# Copy file zip vào Drive
shutil.copy("checkpoints.zip", "/content/drive/My Drive/vietnamese-tts-models/")
print("Đã lưu checkpoints.zip vào Google Drive thành công!")